# k-NN: k Nearest Neighbours

This is a classification algorithm:

![title](http://bdewilde.github.io/assets/images/2012-10-26-knn-concept.png)


For each sample (user) we're testing we pick the nearest k neighbours and classify the sample according to the majority of neighbours.
(e.g. if we chose k=5, and of the 5 chosen users (nearest neighbours) 3 are bots, our user will be classified as bot).

The concept is fairly simple, two things need clarification: how do we choose k and how do we decide which are the nearest neighbours.

### Choosing $k$
The choice can be random at first, bearing in mind 2 things:
1. $k$ should be an odd number, since our set of possible classification values is binary. We might not get majority for an even number.
2. As $k$ grows, it looses sensitivity. However, if it's too small the classification might get skewed. (Helpful, I know)

There are algorithms for optimizing $k$, they result in a range of possible values, from which we can pick one (and hope for the best).

### What does "nearest" mean?

The choice of nearest neighbours can be done in a number of ways. The most widely used one is to calculate some sort of distance between the tested sample and all other samples in the set, and pick the $k$ ones with the shortest distance.

### So how do we turn a bunch of features into comparable objects?

We represent each sample (user) as a vector of numerical values referred to as 'feature vector'. A vector is simply an array, and each cell (index) in the array holds the value of one of our filters, meaning - if we have 5 filters our vectors will be of length 5.
Our filters return values on a very wide scale, some of them return true/false. We need all the values to be normalized meaning have the same range, usually 0-1 (the relative magnitude of the filter's values will remain unchanged, it's just a mapping function).
So the first step will be to normalize all the values. (note: I'll implement this part, it's very straightfoward). 

Once we have the normalized feature vectors for all the samples in our data set, we can start calculating the distances between them. We'll use the most commonly used Euclidean distance.

### Implementation:

#### The distance calculation is as follows:

declare: $totalDistanceSquared = 0$
for each 2 chosen vectors:
    <br>1. iterate over index range:(0 -> length of vector)
    <br>2. calculate distance between the vectors at current index: $(distance)^2 = \left(V_1[i] - V_2[i]\right)^2$
    <br>3. add distance to $totalDistanceSquared$ 
<br>notes: 
    <br>- $V_1$ and $V_2$ are the 2 vectors
    <br>- the index range is 0 to the number of our filters (non-inclusive), or the length of the vectors (same thing)

The distance will be computed for each vector in our training set against all other vectors.
For the purpose of the comparison, we basically create all possible pairs in the set and compute the distance for each pair.
If you get stuck on the algorithm for creating all possible pairs (I did, for quite a while, hope you'll fare better), the solution is the pairing round robin, found here:<br>
https://en.wikipedia.org/wiki/Round-robin_tournament#Scheduling_algorithm
<br>(if you need help implementing it, I have the implementation in Ruby. Remember Ruby?)

The resulting distances should be sorted, and the lowest $k$ results are the $k$ nearest neighbours (lowest because smallest distance).

The majority classification of those $k$ neighbours will decide the classification of our tested vector (sample).

We can see at this point how good the classification is on a very naive basis - new samples will be compared agains our current data set and classified according to it. This means that we have two possible tweaks in order to improve results: different choice of $k$, and increasing data set size.

<br><br>
**This is you guys. Good luck, live long and prosper and thanks for the fish!**
<br><br>

--------------------------------KEEP READING AT YOUR OWN PERIL--------------------------------------------------------

Of course - we, are planning on extending this classification which tends to give not-so-great results, by adding weights to each filter (meaning - we grade each filter based on how indicative it is to the classification).

The addition of the weights gives us the following distance equation:

\begin{align}
totalDistanceSquared = w_0 * \left(V_1[0] - V_2[0]\right)^2 + w_1 * \left(V_1[1] - V_2[1]\right)^2 + w_2 * \left(V_1[2] - V_2[2]\right)^2 + w_3 * \left(V_1[3] - V_2[3]\right)^2 + w_4 * \left(V_1[4] - V_2[4]\right)^2
\end{align}

At which point we call on our optimization algorithm (max/min function), in our case - Gradient Descent, to learn the best weights for our filters, thus eliminating guessing.

Gradient Descent basically gets a "cost function" - a function that represents the difference between the observed (real) classification and the one we calculated  with our... well, distance function?

And here's the pickle - our function is a distance function, and we cannot compare directly pickles to bots. Obviously, there is a widely used solution to this kind of problems (so widely used it took me 3 hours, 32 tabs and some unsettling mumbling to myself) and that is to do what we've already done when we normalized the feature vector - use a mapping function! map the distance values onto another range that better suits our needs. Our range is 0 - 1 (not-bot, bot), so the mapping function should provide that. The function we'll be using is a sigmoid function (logistic function), used in logistic regression (surprise!) and it basically maps all values on x axis (representing the distances we calculated) to (0-1) range on the y axis. Those mapped values are the ones we'll use.
This, as it turns out, gives us the probability that the sample we calculated the distance for is a bot.
<br>**SO**  getting back to our cost function now: it'll calculate the difference between the true classification and the probability we calculated that the tested sample is a bot. So it'll look something like this:

\begin{align}
J(w_1,w_2,w_3,w_4,w_5) = \left(\frac{1}{n}\right) * \left( \sum_{i=1}^n p_i - c_i \right)^2 
\end{align}

(note: sum over all $n$ samples in our data set)

$J$ - cost function (can't remember why its symbol is J)
<br>$n$ - number of samples
<br>$i$ - the i-th sample ranging from 1 to n
<br>$p_i$ - calculated probability for the i-th sample
<br>$c_i$ - real classification for the i-th sample 
<br>$\left(\frac{1}{n}\right)$ at the beggining averages the summed errors.


The more mathematical aspect of the G(radient)D(escent) is the weights adjustment function.

#### GD iterations in broad strokes: ####

<br>we'll set a change rate $r$ (by how much we'd like to modify each weight for the next iteration), and then we'll multiply that rate by the partial derivative of the cost function, for each weight separately (so 5 equations to update per iteration). This will happen at each iteration of GD, and the change to all weights will be applied simultaneously in the following iteration. Why simultaneously is important? (wait for it...)
<br>As a conceptual aside - a derivative is always an idication of change  (e.g. accelaration is the derivative of velocity - it indicates the change in magnitude and direction of the velocity). Meaning - we're calculating the change in the cost function caused by the modification we made to each weight separately (hence the partial part of the derivative).

The weight alteration equation per iteration looks like this:

$w_0 =: w_0 - r * \frac{\partial{J}}{\partial{w_0}}$
<br><br>$w_1 =: w_1 - r * \frac{\partial{J}}{\partial{w_1}}$
<br><br>$w_2 =: w_2 - r * \frac{\partial{J}}{\partial{w_2}}$
<br><br>$w_3 =: w_3 - r * \frac{\partial{J}}{\partial{w_3}}$
<br><br>$w_4 =: w_4 - r * \frac{\partial{J}}{\partial{w_4}}$


note: the $\partial$ stands for partial derivative

As for the why simultaneously - remember that $J$ is the cost function, and if you take a look at it again you'll be reminded that while $n$ and $c$ are numbers, $p$ is actually a more complex expression that links to the distance we calculated. All the weights appear in the equation for $p$ (will show that equation in a sec), so when we calculate $w_1$ for the next iteration for example, we actually assign its *new* value into the $p$ equation that we use in the calculation for $w_2$, and we take the *new* $w_1$ and $w_2$ and plug them into $p$ for $w_3$ etc. So change in each of them affects all the others (chain reaction).

If you want to see where they're plugged in for the next iteration, this is the partial derivative for $w_0$:

<br>\begin{equation*}
\frac{e^ \left(totalDistanceSquared\right)}{\left(1 + e^ \left(totalDistanceSquared\right)\right)^2} * distanceFeature0
\end{equation*}

<br>Reminder: 
<br><br>$distanceFeature0 = \left(V_1[0] - V_2[0]\right)^2$
<br>All other partial derivatives will look the same, only the distance multiplier will be different ($distanceFeature1$ etc).
<br>The part of the equation that actually stores the weights is the $totalDistanceSquared$ function (that you've surely implemented by now) for the **k-NN**:

\begin{align}
totalDistanceSquared = w_0 * \left(V_1[0] - V_2[0]\right)^2 + w_1 * \left(V_1[1] - V_2[1]\right)^2 + w_2 * \left(V_1[2] - V_2[2]\right)^2 + w_3 * \left(V_1[3] - V_2[3]\right)^2 + w_4 * \left(V_1[4] - V_2[4]\right)^2
\end{align}

So when we calculate $w_0$ for the next iteration we then plug it into $totalDistanceSquared$ that appears in $p$ when we calculate $w_1$ etc.

And this is it.

<br>All we have to do is choose $r$ (the change rate) - meaning, how big are the changes in each iteration (too big we might miss the real minimum, too small GD will run a very long time), and what is the stopping point for us - how low would we like GD to go before we pull the plug (every new iteration the first thing GD does is to re-evaluate the cost function with the new weights and get the new cost. If it's under our chosen stop point, it'll stop and return the calculated weights).

So we basically decide what is the satisfactory probability for us that a bot is a bot (0.8? 0.92?), and set the stop-point (to be 0.2 or 0.08) respectively.

Hope this makes some sense, if not, well, you know what you can do...
